In [3]:
import cv2
import numpy as np
from scipy.signal import medfilt

In [4]:
def luminance_diff(frame1, frame2):
  y1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2YUV)[:, :, 0]
  y2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2YUV)[:, :, 0]
  return abs(np.mean(y1) - np.mean(y2))

In [9]:
def detect_scene(video_path, step=3, threshold=2.0, smooth_kernel=5):
  cap = cv2.VideoCapture(video_path)
  fps = cap.get(cv2.CAP_PROP_FPS)
  prive_frame = None
  diffs = []
  frames = []

  while cap.isOpened():
    ref, frame = cap.read()
    if not ref:
      break
    frames.append(frame)
  cap.release()

  if len(frames) <= step:
    return [], fps

  for i in range(len(frames) - step):
    d = luminance_diff(frames[i], frames[i + step])
    diffs.append(d)

  diffs = np.array(diffs)

  if len(diffs) >= smooth_kernel and smooth_kernel > 1:
    diffs_smooth = medfilt(diffs, kernel_size=smooth_kernel)
  else:
    diffs_smooth = diffs
  threshold_new = np.mean(diffs_smooth) + threshold * np.std(diffs_smooth)

  scene_change = []
  for i, d in enumerate(diffs_smooth):
    if d > threshold_new:
      scene_change.append(i + step // 2 + 1)
  return scene_change, fps

In [10]:
scene_change, fps = detect_scene('/content/drive/MyDrive/practical_data/man_in_costume.mp4', threshold=2.0)
print('Кадры смены сцен', scene_change)

Кадры смены сцен [46, 47, 48, 245, 246, 247, 296, 297, 298, 367, 368, 369]


In [21]:
scene_change, fps = detect_scene('/content/drive/MyDrive/practical_data/man_in_costume.mp4', threshold=2.0, step=3, smooth_kernel=3)
print('Кадры смены сцен', scene_change)

Кадры смены сцен [46, 47, 48, 189, 190, 245, 246, 247, 296, 297, 298, 367, 368, 369]
